In [ ]:
# Scripts to do:
# - Refresher
# - Mapper

In [107]:
###############################################################################################################
############################### Correction of mapping_finish ##################################################
###############################################################################################################

import pandas as pd
from bycon import *
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient()
db = client.progenetix
bs = db.biosamples

# Read the data frame
# df = pd.read_csv("temp/mapfile.tsv", sep='\t', header=0, low_memory = False)
df = pd.read_csv("../temp/mapfile.tsv", sep='\t', header=0, low_memory = False)

# Create legacy ids / external references
df['case_id'] = 'pgx:TCGA-' + df['case_id']
df['sample_id'] = 'pgx:TCGA-' + df['sample_id']


# Generate callset_ids per aliquot for import
callset_ids = []
for aliquot in set(df['aliquot_id']):
    cs_id = generate_id('pgxcs')
    df.loc[df['aliquot_id'] == aliquot, 'callset_id'] = cs_id

# Create placeholder for variant id, gets created while import
df['variant_id'] = ['pgxvar-'] * len(df)

# Create sets of ids for fast mapping
sid = set(df['sample_id'])

# Map sample_id and get biosample_id
for i in sid:
    # If there are more than one document, something is wrong
    if bs.count_documents({"external_references.id": {'$regex': i},"biosample_status.id":"EFO:0009656"}) > 1:
        print('More than one document found.')
        
    else:
        hit = bs.find({"external_references.id": {'$regex': i},"biosample_status.id":"EFO:0009656"})
        for entry in hit:
            df.loc[df['sample_id'] == i, 'biosample_id'] = entry['id']
            df.loc[df['sample_id'] == i, 'individual_id'] = entry['individual_id']

# alternate_bases_1 is the same as reference_bases
df['alternate_bases'] = df['alternate_bases_2']

# Clean up
df = df[['biosample_id', 'variant_id', 'callset_id', 'chromosome', 'start', 'end', 'strand',
        'reference_bases', 'alternate_bases', 'hgvsc', 'variant_classification', 'variant_type', 
        'hgvsp', 'hgvsp_short', 'aliquot_id', 'reference_id', 'case_id', 'sample_id']]

# Write finished mapping file
os.makedirs('temp/', exist_ok = True) # Check for the directory
df.to_csv('temp/mappingfile.tsv', sep = '\t', index = False)  # and create .tsv file in the directory

In [123]:
df['biosample_id'] = ['dummy'] * len(df)

In [151]:
ls = df.loc[df['variant_type'] == 'SNP', 'hgvsc']
ll = df.loc[df['variant_type'] == 'SNP', 'reference_bases']

for i in ls:
    i = i.split()

c.1760C>G G
c.103G>C G
c.658A>T A
c.1380C>A C
c.3484C>A C
c.2510T>A T


KeyError: 6

In [ ]:
###############################################################################################################
##################################### Initialisation ##########################################################
###############################################################################################################

In [70]:
import os
from pymongo import MongoClient
import bycon
import pandas as pd
import time, base36

# Prepare mongodb tools
client = MongoClient()
db = client.progenetix
bs = db.biosamples

In [ ]:
###############################################################################################################
##################################### COSMIC #############################################################
###############################################################################################################
## Problems
mutant_export_consensus = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicMutantExportCensus.tsv', sep = '\t', low_memory = False)
mutant_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicMutantExport.tsv', sep = '\t', low_memory = False)
complete_target_screens = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicCompleteTargetedScreensMutantExport.tsv', sep = '\t', low_memory = False)

In [81]:
resistance_mutations = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicResistanceMutations.tsv', sep = '\t', low_memory = False)

In [82]:
ncv = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicNCV.tsv', sep = '\t', low_memory = False)

In [87]:
genome_screens = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicGenomeScreensMutantExport.tsv', sep = '\t', low_memory = False)

In [88]:
fusion_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicFusionExport.tsv', sep = '\t', low_memory = False)

In [89]:
breakpoints_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicBreakpointsExport.tsv', sep = '\t', low_memory = False)

In [84]:
struct_export = pd.read_csv('/Users/kayvongrunigen/Downloads/CosmicStructExport.tsv', sep = '\t', low_memory = False)

In [92]:
resistance_mutations = resistance_mutations['Pubmed Id']
ncv = ncv['PUBMED_PMID']
genome_screens = genome_screens['Pubmed_PMID']
fusion_export = fusion_export['PUBMED_PMID']
breakpoints_export = breakpoints_export['Pubmed_PMID']
struct_export = struct_export['PUBMED_PMID']

In [103]:
cosmic = [resistance_mutations, ncv, genome_screens,
          fusion_export, breakpoints_export, struct_export]

In [105]:
cosmic

[0       12623848
 1       15685537
 2       27432227
 3       27432227
 4       24055055
           ...   
 5094    30914635
 5095    31575540
 5096    23344087
 5097    24185512
 5098    31697823
 Name: Pubmed Id, Length: 5099, dtype: int64,
 0                  NaN
 1           26551670.0
 2                  NaN
 3                  NaN
 4                  NaN
                ...    
 18466904           NaN
 18466905           NaN
 18466906    29109526.0
 18466907           NaN
 18466908           NaN
 Name: PUBMED_PMID, Length: 18466909, dtype: float64,
 0                  NaN
 1           33021035.0
 2           31336886.0
 3                  NaN
 4           33024263.0
                ...    
 46462899           NaN
 46462900           NaN
 46462901           NaN
 46462902           NaN
 46462903           NaN
 Name: Pubmed_PMID, Length: 46462904, dtype: float64,
 0        20033038
 1        20033038
 2        20033038
 3        20033038
 4        20033038
            ...   
 86501

In [97]:
cleaned_cosmic = [x for x in cosmic if str(x) != 'nan']

[[14618630,
  22368270,
  24457237,
  23799845,
  25712683,
  22773810,
  17020982,
  21921847,
  28274743,
  24555578,
  19925053,
  28235842,
  31758409,
  30914635,
  20367437,
  31371345,
  28434515,
  20871262,
  24463458,
  23392356,
  31668326,
  17332333,
  24729716,
  21383288,
  30711944,
  31254668,
  23969938,
  26648736,
  21264552,
  15728811,
  14645423,
  30036146,
  26413254,
  12949711,
  20682976,
  22722787,
  27666659,
  32999653,
  28049639,
  26130666,
  28768491,
  29106415,
  21895409,
  25137394,
  27304188,
  17008892,
  28522754,
  22722830,
  26308885,
  22722843,
  22354205,
  27257132,
  22071596,
  24942894,
  22108465,
  24105277,
  24813888,
  17973572,
  25735500,
  24736079,
  19155283,
  19181917,
  32694622,
  30675302,
  30089600,
  19915144,
  22718859,
  33218975,
  22993320,
  28271038,
  23433665,
  24265153,
  24265155,
  24265154,
  19726788,
  21221851,
  31543779,
  27929064,
  19589612,
  30683630,
  21623281,
  12204532,
  23912954,
  23

In [4]:
###############################################################################################################
##################################### Diagnostics #############################################################
###############################################################################################################

In [69]:
# Read in dataframe from intermediate and mapping file
df = pd.read_csv('../temp/intermediate_mapping_file.csv') ###### Make input later

hits_c = []

for ids in import_template['case_id']:
    n_hits_c = bs.count_documents({"external_references.id": ids,"biosample_status.id":"EFO:0009656"})
    hits_c.append(n_hits_c)
    
single_hit = 0
multiple_hits = 0
no_hit = 0

for i in hits_c:
    if i == 1:
        single_hit += 1
    if i == 0:
        no_hit += 1
    if i != 0 and i != 1:
        multiple_hits += 1

print("Single hits:", single_hit, "\nMultiple hits:", multiple_hits,
      "\nNo hit:", no_hit, "\nTotal:", len(hits_c))
# Single hits: 9775 
# Multiple hits: 125 
# No hit: 208 
# Total: 10108

# Legacy ids = sample id


hits_s = []

for ids in import_template['biosample_id']:
    n_hits_s = bs.count_documents({"external_references.id": ids,"biosample_status.id":"EFO:0009656"})
    hits_s.append(n_hits_s)

    single_hit = 0
multiple_hits = 0
no_hit = 0

for i in hits_s:
    if i == 1:
        single_hit += 1
    if i == 0:
        no_hit += 1
    if i != 0 and i != 1:
        multiple_hits += 1

print("Sample Hits:\n", "Single hits:", single_hit, "\nMultiple hits:", multiple_hits,
      "\nNo hit:", no_hit, "\nTotal:", len(hits_c))

Single hits: 2480711 
Multiple hits: 54429 
No hit: 34950 
Total: 2570090


In [ ]:
# Findings!
# The sample ids have hierarchy:
# Case UUID: becedbfd-b2aa-4dde-b7f4-29e6f59ec32c in data bank
# Sample UUID: f6ee8148-9fb7-404e-b951-d1af8d06974f not in data bank
# Belongs to sample TCGA-E2-A1IK-01A
# Sample UUID: 8577ac01-1274-4bd5-ab04-380eaa78d95b in data bank
# Belongs to aliquot TCGA-E2-A1IK-01A-11D-A17G-09
# TCGA-E2-A1IK-01A
#     Portions
#     TCGA-E2-A1IK-01A-11
#         Analytes
#         TCGA-E2-A1IK-01A-11D
#             Aliquots
#             TCGA-E2-A1IK-01A-11D-A141-01
#             TCGA-E2-A1IK-01A-11D-A17G-09
#             TCGA-E2-A1IK-01A-11D-A140-02
#             TCGA-E2-A1IK-01A-11D-A142-09
#             TCGA-E2-A1IK-01A-11D-A145-05

# Solution: find sample UUID of aliquot sample UUID -> R package TCGAutils


# COSMIC is in TCGA
# 118 - COSMIC	Overlapping COSMIC variants


In [ ]:
###############################################################################################################
##################################### Other Scripts ###########################################################
###############################################################################################################

In [10]:
# mapTCGAids.py
#!/usr/local/bin/python3

from pymongo import MongoClient
import argparse
import json
import re
from progress.bar import Bar

################################################################################
################################################################################
################################################################################

def _get_args():

    parser = argparse.ArgumentParser()
    parser.add_argument("-t", "--test", help="test setting")
    parser.add_argument('-i', '--inputfile', help='a custom file to specify input data')
    parser.add_argument('-s', '--survivalfile', help='...')

    return parser.parse_args()

################################################################################

def main():

    args = _get_args()

    ds_id = 'progenetix'
    
    mapTCGAids(args, ds_id)
    addTCGAsurvival(args, ds_id)

################################################################################

def mapTCGAids(args, ds_id): #re-construct the external descriptions, and add submitter and case ids  of TCGA sampels

    if not args.inputfile:
        print( "Not updating ids - you have to provide an inputfile -i")
        return

    data_client = MongoClient('localhost', 27017)
    data_db = data_client[ ds_id ]
    bios_coll = data_db[ "biosamples" ]
    ind_coll = data_db[ "individuals" ]

    ind_ids = {}

    fh=open(args.inputfile, 'r')
    idlist = [line.rstrip() for line in fh.readlines()[1:]]
    fh.close()

    no = len(idlist)
    if not args.test:
        bar = Bar("{} samples".format(no), max = no, suffix='%(percent)d%%'+" of "+str(no) )
    else:
        idlist = idlist[0:20]

    for line in idlist:
        tcga_s_id, tcga_c_id, tcga_submitter_id  = line.rstrip().split('\t')

        q = {'info.legacy_ids': tcga_s_id}
        b = bios_coll.find_one(q)

        i_e_r = [
            {"id": "pgx:TCGA-"+tcga_c_id, "label": "TCGA case_id"},
            {"id": "pgx:TCGA-"+tcga_submitter_id, "label": "TCGA submitter_id"}
        ]

        if tcga_c_id in ind_ids:
            ind_ids[tcga_c_id]["biosample_ids"].append(b["id"])
        else:
            ind_ids.update({
                tcga_c_id: {
                    "external_references": i_e_r,
                    "biosample_ids": [b["id"]]
                }
            })

        b_e_r = i_e_r.copy()
        b_e_r.append({"id": "pgx:TCGA-"+tcga_s_id, "label": "TCGA sample_id"})

        for e in b["external_references"]:
            if not "TCGA" in e["id"]: # <= important to keep other ers
                b_e_r.append(e)
            if re.match(r'^(?:pgx:)?TCGA\-\w+?$', e["id"]):
                project = re.sub('TCGA-', '', e["id"])
                project = re.sub('pgx:', '', project)
                b_e_r.append({"id": "pgx:TCGA-"+project, "label": "TCGA "+project+" project"})

        if args.test:
            print(tcga_c_id, ind_ids[tcga_c_id]["biosample_ids"])
        else:
            bar.next()
            bios_coll.update_one({"_id":b["_id"]}, {"$set": {"external_references": b_e_r}})


    if not args.test:
        bar.finish()

    ############################################################################

    no = len(ind_ids.keys())
    if not args.test:
        bar = Bar("{} individuals".format(no), max = no, suffix='%(percent)d%%'+" of "+str(no) )

    for i_id, i_data in ind_ids.items():

        i_e_r = i_data["external_references"]
        i_e_r_ids = set()

        ind_ids = bios_coll.distinct("individual_id", {"id":{"$in":i_data["biosample_ids"]}})

        i_s = ind_coll.find({"id":{"$in":ind_ids}})
        for i in i_s:
            for e in i["external_references"]:
                if not "TCGA" in e["id"]: # <= important to keep other ers
                    if not e["id"] in i_e_r_ids:
                        i_e_r_ids.add(e["id"])
                        i_e_r.append(e)

        ind_coll.update_many({"id":{"$in":ind_ids}}, {"$set":{"external_references":i_e_r}})

        if len(ind_ids) > 1:
            ind_ids = ind_ids[1:]

            print("deleting {} of {} individuals".format(len(ind_ids), len(i_data["biosample_ids"])))

            ind_coll.delete_many({"id":{"$in":ind_ids}})


# def mapTCGAsurvival(clinical,output):# map survival information to TCGA samples
#     with open(output + '.csv', 'w') as f:
#         f.write('case_id')
#         f.write('\t')
#         f.write('vital_status')
#         f.write('\n')
#     client = MongoClient('localhost', 27017)
#     #db = client[database]
#     file = open(clinical + '.json', )
#     data = json.load(file)
#     for i in data:
#         with open(output + '.csv', 'a+') as f:
#             f.write(i['case_id'])
#             f.write('\t')
#             f.write(i['demographic']['vital_status'])
#             f.write('\n')

def addTCGAsurvival(args, ds_id):# add survival information for cancer TCGA samples

    if not args.survivalfile:
        print( "You have to provide an survivalfile -s")
        return

    data_client = MongoClient('localhost', 27017)
    data_db = data_client[ ds_id ]
    bios_coll = data_db[ "biosamples" ]
    ind_coll = data_db[ "individuals" ]

    fh=open(args.survivalfile, 'r')
    survlist = [line.rstrip() for line in fh.readlines()[1:]]
    fh.close()

    no = len(survlist)
    if not args.test:
        bar = Bar("{} samples".format(no), max = no, suffix='%(percent)d%%'+" of "+str(no) )
    else:
        survlist = survlist[0:20]

    for line in survlist:
        temp = line.split('\t')
        e_r_s_id = "pgx:TCGA-"+temp[0]
        surv = temp[1].lower()
        bs_q = {'external_references.id': e_r_s_id,'cohorts.id':'pgx:cohort-TCGAcancers'}
        if args.test:
            print(e_r_s_id, surv)
        else:
            bar.next()
            bios_coll.update_many(bs_q, {'$set': {'info.survival_status': surv}})

    if not args.test:
        bar.finish()
        

################################################################################
################################################################################
################################################################################

if __name__ == '__main__':
    main()

#mapTCGAids('TCGA_ids.txt','progenetix')
#addTCGAsurvival('TCGAcase_survival','progenetixCopy')

usage: ipykernel_launcher.py [-h] [-t TEST] [-i INPUTFILE] [-s SURVIVALFILE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/kayvongrunigen/Library/Jupyter/runtime/kernel-ce3c82a0-b671-426e-b557-5e0b2a949c95.json


SystemExit: 2

/opt/homebrew/mambaforge/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#tcga_mapper.py
#!/usr/local/bin/python3

from pymongo import MongoClient
import json
import re
import json
import argparse
from os import path

################################################################################

def _get_args():

    parser = argparse.ArgumentParser()

    parser.add_argument("-d", "--datasetid", help="dataset, usually progenetix")
    parser.add_argument('-t', '--tcgafile', help='TCGA json data file')
    parser.add_argument('-i', '--idfile', help='idfile')
    parser.add_argument('-u', '--update', help='update modus')

    return parser.parse_args()

################################################################################

def main():

	args = _get_args()

	update_TCGA_data(args)

################################################################################

def update_TCGA_data(args):

	if not args.datasetid:
		print("no datasetid was specified with -d")
		exit()

	if not args.idfile:
		print("no id file was specified with -i")
		exit()

	if not args.tcgafile:
		print("no tcga file was specified with -t")
		exit()

	if not args.update:
		print('no update will be performed - activate with "-u 1"')

	mongo_client = MongoClient()
	data_db = mongo_client[args.datasetid]
	bios_coll = data_db["biosamples"]
	ind_coll = data_db["individuals"]

	f = open(args.idfile, 'r+')
	id_lines = [line for line in f.readlines()]
	f.close()

	tcga_ids = {}

	for id_l in id_lines:
		id_l = id_l.rstrip()
		sample_id, case_id, submitter_id = re.split("\t", id_l)
		tcga_ids.update({sample_id:{"sample_id": sample_id, "case_id": case_id, "submitter_id": submitter_id }})

	with open(args.tcgafile, 'r') as c_f:
		c = c_f.read()
		c_l = json.loads(c)

	tcga_clin = {}

	for tcga_c in c_l:
		tcga_clin.update({ tcga_c["case_id"]: tcga_c })

	for s_id, s_id_o in tcga_ids.items():

		s = bios_coll.find_one({"info.legacy_id": s_id })
		if not s:
			print('!!! no biosample for "{}"'.format(s_id))
			continue

		c_id = s_id_o["case_id"]

		if not c_id in tcga_clin:
			print('!!! no tcga data for case_id "{}"'.format(c_id))
			continue

		clin = tcga_clin[c_id]

		bios_update = {
			"external_references": [],
			"info": s.get("info", {})     # <=== important!
		}

		# references
		# first collecting all "non-TCGA ones" for keeping, then adding the new TCGA ones
		for pgx_e_r in s["external_references"]:
			if "TCGA" in pgx_e_r["label"]:
				if not "project" in pgx_e_r["label"]:
					# skipping previous TCGA labels _except_ the TCGA-XXX for projects
					continue
			pgx_e_r.pop("description", None)
			bios_update["external_references"].append(pgx_e_r)

			for id_t in s_id_o.keys():
				n_id = s_id_o[id_t]
				if not "TCGA-" in n_id:
					n_id = "TCGA-"+s_id_o[id_t]
				id_obj = { "id": n_id, "label": "TCGA "+id_t }
				bios_update["external_references"].append(id_obj)

		# clinical
		f_u_s = clin["demographic"].get("vital_status", "").lower()
		if "dead" in f_u_s:
			bios_update.update({"followup_state":{"id":"EFO:0030049", "label":"dead (follow-up status)"}})
			bios_update["info"].update({"survival_status":"dead"})
		elif "alive" in f_u_s:
			bios_update.update({"followup_state":{"id":"EFO:0030041", "label":"alive (follow-up status)"}})
			bios_update["info"].update({"survival_status":"alive"})
        #Ziying update => since ["demographic"]["days_to_death"] should be correct for dead samples
        if clin["demographic"]["days_to_death"]:
            f_u_d = clin["demographic"]["days_to_death"]
        else:
            f_u_d = clin["diagnoses"][0]["days_to_last_follow_up"]
		#f_u_d = clin["diagnoses"][0]["days_to_last_follow_up"]
		if isinstance(f_u_d, int):
			f_u_m = round(f_u_d/30.5)
			bios_update["info"].update({"followup_months":f_u_m})

		a_d = clin["diagnoses"][0]["age_at_diagnosis"]
		if isinstance(a_d, int):
			a = _d2iso(a_d)
			bios_update.update({"individual_age_at_collection":"{}".format(a)})

		if args.update:
			bios_coll.update_one({"_id": s["_id"] }, { '$set': bios_update } )
		else:
			print(bios_update)

		########################################################################
		# individuals
		########################################################################

		i = ind_coll.find_one({"id": s["individual_id"] })
		if not s:
			print('!!! no individual for "{}"'.format(s["individual_id"]))
			continue

		ind_update = {
			"external_references": [],
			"info": i.get("info", {}),
		}

		# references
		for pgx_e_r in s["external_references"]:
			if "TCGA" in pgx_e_r["label"]:
				if "biosample" in pgx_e_r["label"]:
					continue
				if "TCGA collection" in pgx_e_r["label"]:
					continue
			pgx_e_r.pop("description", None)
			bios_update["external_references"].append(pgx_e_r)
			ind_update["external_references"].append(pgx_e_r)

			for id_t in s_id_o.keys():
				id_obj = { "id": "TCGA-"+s_id_o[id_t], "label": "TCGA "+id_t }
				bios_update["external_references"].append(id_obj)
				ind_update["external_references"].append(id_obj)

		sex = clin["demographic"].get("gender", "")
		if "female" in sex:
			ind_update.update({"sex":{"id":"PATO:0020002", "label":"female genotypic sex"}})
		elif "male" in sex:
			ind_update.update({"sex":{"id":"PATO:0020001", "label":"male genotypic sex"}})

		if args.update:
			ind_coll.update_one({"_id": i["_id"] }, { '$set': ind_update } )
		else:
			print(ind_update)

################################################################################

def _d2iso(d):

	y = int(d / 365.25)
	m = int((d % 365.25) / 30.5)
	d = int(d - (y * 365.25 + m * 30.5))

	return "P{}Y{}M{}D".format(y,m,d)

################################################################################
################################################################################

if __name__ == '__main__':
    main(  )

    
################################################################################
################################################################################
################################################################################
################################################################################
    
# tcga_mapperZY.py    
    
    
    
    
    
    
    
    #!/usr/local/bin/python3

from pymongo import MongoClient
import json
import re
import json
import argparse
from os import path

################################################################################

def _get_args():

    parser = argparse.ArgumentParser()

    parser.add_argument("-d", "--datasetid", help="dataset, usually progenetix")
    parser.add_argument('-t', '--tcgafile', help='TCGA json data file')
    parser.add_argument('-i', '--idfile', help='idfile')
    parser.add_argument('-u', '--update', help='update modus')

    return parser.parse_args()

################################################################################

def main():

	args = _get_args()

	update_TCGA_data(args)

################################################################################

def update_TCGA_data(args):

	if not args.datasetid:
		print("no datasetid was specified with -d")
		exit()

	if not args.idfile:
		print("no id file was specified with -i")
		exit()

	if not args.tcgafile:
		print("no tcga file was specified with -t")
		exit()

	if not args.update:
		print('no update will be performed - activate with "-u 1"')

	mongo_client = MongoClient()
	data_db = mongo_client[args.datasetid]
	bios_coll = data_db["biosamples"]
	ind_coll = data_db["individuals"]

	f = open(args.idfile, 'r+')
	id_lines = [line for line in f.readlines()]
	f.close()

	tcga_ids = {}

	for id_l in id_lines:
		id_l = id_l.rstrip()
		sample_id, case_id, submitter_id = re.split("\t", id_l)
		tcga_ids.update({sample_id:{"sample_id": sample_id, "case_id": case_id, "submitter_id": submitter_id }})

	with open(args.tcgafile, 'r') as c_f:
		c = c_f.read()
		c_l = json.loads(c)

	tcga_clin = {}

	for tcga_c in c_l:
		tcga_clin.update({ tcga_c["case_id"]: tcga_c })

	for s_id, s_id_o in tcga_ids.items():

		s = bios_coll.find_one({"info.legacy_id": s_id })
		if not s:
			print('!!! no biosample for "{}"'.format(s_id))
			continue

		c_id = s_id_o["case_id"]

		if not c_id in tcga_clin:
			print('!!! no tcga data for case_id "{}"'.format(c_id))
			continue

		clin = tcga_clin[c_id]

		bios_update = {
			#"external_references": [],
			"info": s.get("info", {})     # <=== important!
		}

		# references
		#first collecting all "non-TCGA ones" for keeping, then adding the new TCGA ones
		# for pgx_e_r in s["external_references"]:
		# 	if "TCGA" in pgx_e_r["label"]:
		# 		if not "project" in pgx_e_r["label"]:
		# 			# skipping previous TCGA labels _except_ the TCGA-XXX for projects
		# 			continue
		# 	pgx_e_r.pop("description", None)
		# 	bios_update["external_references"].append(pgx_e_r)
		#
		# 	for id_t in s_id_o.keys():
		# 		n_id = s_id_o[id_t]
		# 		if not "TCGA-" in n_id:
		# 			n_id = "TCGA-"+s_id_o[id_t]
		# 		id_obj = { "id": n_id, "label": "TCGA "+id_t }
		# 		bios_update["external_references"].append(id_obj)

		# clinical
		f_u_s = clin["demographic"].get("vital_status", "").lower()
		if "dead" in f_u_s:
			bios_update.update({"followup_state":{"id":"EFO:0030049", "label":"dead (follow-up status)"}})
			bios_update["info"].update({"survival_status":"dead"})
		elif "alive" in f_u_s:
			bios_update.update({"followup_state":{"id":"EFO:0030041", "label":"alive (follow-up status)"}})
			bios_update["info"].update({"survival_status":"alive"})
		if "days_to_death" in clin["demographic"]:
			f_u_d = clin["demographic"]["days_to_death"]
		else:
			f_u_d = clin["diagnoses"][0]["days_to_last_follow_up"]

		#update info clinical
		if "ajcc_clinical_stage" in clin["diagnoses"][0]:
			bios_update["info"].update({"tumor_stage":clin["diagnoses"][0]["ajcc_clinical_stage"].split(' ')[1]})
		if "tumor_grade" in clin["diagnoses"][0]:
			bios_update["info"].update({"tumor_grade": clin["diagnoses"][0]["tumor_grade"]})
		t=n=m=''
		if "ajcc_pathologic_t" in clin["diagnoses"][0]:
			t = clin["diagnoses"][0]["ajcc_pathologic_t"]
		if "ajcc_pathologic_n" in clin["diagnoses"][0]:
			t = clin["diagnoses"][0]["ajcc_pathologic_n"]
		if "ajcc_pathologic_m" in clin["diagnoses"][0]:
			t = clin["diagnoses"][0]["ajcc_pathologic_m"]
		tnm = t+n+m
		if tnm:
			bios_update["info"].update({"tnm": tnm})




		if isinstance(f_u_d, int):
			f_u_m = round(f_u_d/30.5)
			bios_update["info"].update({"followup_months":f_u_m})

		a_d = clin["diagnoses"][0]["age_at_diagnosis"]
		if isinstance(a_d, int):
			a = _d2iso(a_d)
			bios_update.update({"individual_age_at_collection":"{}".format(a)})

		if args.update:
			bios_coll.update_one({"_id": s["_id"] }, { '$set': bios_update } )
		else:
			print(bios_update)

		########################################################################
		# individuals
		########################################################################

		i = ind_coll.find_one({"id": s["individual_id"] })
		if not s:
			print('!!! no individual for "{}"'.format(s["individual_id"]))
			continue

		ind_update = {
			"external_references": [],
			"info": i.get("info", {}),
		}

		# references
		for pgx_e_r in s["external_references"]:
			if "TCGA" in pgx_e_r["label"]:
				if "biosample" in pgx_e_r["label"]:
					continue
				if "TCGA collection" in pgx_e_r["label"]:
					continue
			pgx_e_r.pop("description", None)
			#bios_update["external_references"].append(pgx_e_r)
			ind_update["external_references"].append(pgx_e_r)

			for id_t in s_id_o.keys():
				id_obj = { "id": "TCGA-"+s_id_o[id_t], "label": "TCGA "+id_t }
				#bios_update["external_references"].append(id_obj)
				ind_update["external_references"].append(id_obj)

		sex = clin["demographic"].get("gender", "")
		if "female" in sex:
			ind_update.update({"sex":{"id":"PATO:0020002", "label":"female genotypic sex"}})
		elif "male" in sex:
			ind_update.update({"sex":{"id":"PATO:0020001", "label":"male genotypic sex"}})

		#update diseases in individuals


		if args.update:
			ind_coll.update_one({"_id": i["_id"] }, { '$set': ind_update } )
		else:
			print(ind_update)

################################################################################

def _d2iso(d):

	y = int(d / 365.25)
	m = int((d % 365.25) / 30.5)
	d = int(d - (y * 365.25 + m * 30.5))

	return "P{}Y{}M{}D".format(y,m,d)

################################################################################
################################################################################

if __name__ == '__main__':
    main(  )